In [39]:
# Dependencies
import requests
import json
from pprint import pprint
import pandas as pd
import os
import csv
import numpy as np
import time 
import tweepy
from config import consumer_key, consumer_secret, access_token, access_token_secret
from datetime import datetime
import math
import pytz
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


utc=pytz.UTC
startDate = datetime(2018, 1, 1, 0, 0, 0)
endDate = datetime(2018, 4, 10, 0, 0, 0)
startDate = utc.localize(startDate) 
endDate = utc.localize(endDate) 

In [40]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

path2 = "resources/npr_stations_twitter_by_county.csv"
df_NPR_raw = pd.read_csv(path2, encoding = "latin1")

df_NPR_raw = df_NPR_raw[df_NPR_raw['Twitter Handle'] != 'None']

print(df_NPR_raw.count())
df_NPR=df_NPR_raw.drop_duplicates(subset= ["Twitter Handle"])
print(df_NPR.count())

df_tweet = pd.DataFrame(data=df_NPR)
df_tweet.head()

Unnamed: 0            700
FIPS                  700
County Name           700
City                  700
State                 700
Latitude              700
Longitude             700
Station Identifier    700
Channel Number        700
AM/FM                 700
Twitter Handle        700
Alternate              11
FIPS_block            700
dtype: int64
Unnamed: 0            432
FIPS                  432
County Name           432
City                  432
State                 432
Latitude              432
Longitude             432
Station Identifier    432
Channel Number        432
AM/FM                 432
Twitter Handle        432
Alternate               8
FIPS_block            432
dtype: int64


,Unnamed: 0,FIPS,County Name,City,State,Latitude,Longitude,Station Identifier,Channel Number,AM/FM,Twitter Handle,Alternate,FIPS_block
0,651,1015,Calhoun,Jacksonville,Alabama,33.813713,-85.761354,WLJS,91.9,FM,@WLJS_919,NaN,10150021012089
1,412,1033,Colbert,Muscle Shoals,Alabama,34.744811,-87.667529,WQPR,88.7,FM,@Alpublicradio,NaN,10330207042005
3,603,1055,Etowah,Gadsden,Alabama,34.014264,-86.006639,WSGN,91.5,FM,@WBHM903,NaN,10550012001034
4,593,1069,Houston,Dothan,Alabama,31.223231,-85.390489,WRWA,88.7,FM,@troypublicradio,NaN,10690406001032
5,602,1073,Jefferson,Birmingham,Alabama,33.520661,-86.802490,WBHM,90.3,FM,@WBHM,NaN,10730027001097


In [45]:
# "Real Person" Filters
min_tweets = 5
max_tweets = 500000
max_followers = 20000
max_following = 20000
lang = "en"

# Set other parameters
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify= True, parser=tweepy.parsers.JSONParser())
entry = 0
unique_ids = []
oldest_tweet = None
output_list = pd.DataFrame(columns=['created_at','account_location','text_tweet', 'name'])

In [46]:
# Super Twitter Loop
for (idx, row) in df_tweet.iterrows():
    #entry += 1 
    user_name = (row.loc['Twitter Handle'])
    # Loop through each user
    counter = 0
    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []
    converted_timestamps = []
    tweet_times = []
    counter_list = [] 
    # Loop through pages.
    for x in range(30):
        # Get all tweets from home feed
        public_tweets = api.user_timeline(user_name, page=x, tweet_mode="extended", count=100)
        #try:
        # Loop through all tweets
        for tweet in public_tweets:
            pprint(tweet)
        #Use filters to check if user meets conditions
            if (tweet['user']["followers_count"] < max_followers and
                tweet['user']["statuses_count"] > min_tweets and
                tweet['user']["statuses_count"] < max_tweets and
                tweet['user']["friends_count"] < max_following and
                tweet["lang"] == lang):
        #Read information
                    created_at= tweet['created_at']
                    created_at =datetime.strptime(created_at, "%a %b %d %H:%M:%S %z %Y")
                    name = tweet['user']['screen_name']                            
                    pprint(created_at)
                    account_location = tweet['user']["location"]
                    pprint(account_location)
                    text_tweet = tweet["full_text"]
                    pprint(text_tweet)
                    output_list.at[idx, 'created_at'] = created_at
                    output_list.at[idx, 'text_tweet'] = text_tweet
                    output_list.at[idx, 'account_location'] = account_location
                    output_list.at[idx, 'name'] = name
#         except Exception as e:
#            print(e)
#            time.sleep(5)
#            print("restarting...")
#            pass

{'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Jun 28 22:32:37 +0000 2016',
 'display_text_range': [0, 125],
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [{'display_url': 'instagram.com/p/BHNwzjgBtYw/',
                        'expanded_url': 'https://www.instagram.com/p/BHNwzjgBtYw/',
                        'indices': [102, 125],
                        'url': 'https://t.co/cSBcRPXRqJ'}],
              'user_mentions': []},
 'favorite_count': 1,
 'favorited': False,
 'full_text': "June orientation dates are done &amp; we've had a lot of "
              'interest in the student media so yay 💃🏻 see… '
              'https://t.co/cSBcRPXRqJ',
 'geo': None,
 'id': 747920669195698178,
 'id_str': '747920669195698178',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None

KeyboardInterrupt: 

In [47]:
output_list.head(200)

,created_at,account_location,text_tweet,name
0,2016-05-31 20:49:30+00:00,"Jacksonville, AL",Our last account kept getting hacked so we had...,WLJS_919
1,2018-01-15 17:05:09+00:00,Alabama,You'll hear Danny Glover tomorrow on APR for t...,ALPublicRadio


In [ ]:
output_test = output_list
#.dropna(subset=['place_name_long'])

output_test.count()